[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zhimingkuang/Harvard-AM-115/blob/main/05_model_fitting/fit_yeast_ode.ipynb)

## This code fits the yeast data to the logistic equation

It gives an example of how to read in the csv data and provides a template for the exercise of modeling running.

In [5]:
import pandas as pd
import numpy as np
from scipy.integrate import ode
import matplotlib.pylab as plt

In [6]:
# This code gives an example of how to read in the csv data and a template
# for the workshop

# change this to the path for your data
datapath='./';

# read in the state ID and state population
PopulationTable=pd.read_csv(datapath+'US_states_population.csv');

# choose a state
stateid=544

rowindex=PopulationTable.LocationID==stateid; #find the corresponding row
PopulationTable.StateName[rowindex] # state name
N=PopulationTable.Population[rowindex]; # total state population

In [7]:
# Read the data for a given state
def read_covid_data(filename,stateid):
    # read in the data as a table
    T=pd.read_csv(filename);
    rows=T.location_id==stateid;
    stateT=T[rows][['date','mean']]
    # Turn into a time table
#     data = table2timetable( stateT, 'RowTimes','Var1');
    stateT['date']=pd.to_datetime(stateT['date'])
    stateT=stateT.set_index('date')
    return stateT

# Creating a function equivalent to ode45 in Matlab
def ode45(f,tint,y0,rtol=1e-9):
    r = ode(f).set_integrator('zvode', method='bdf',rtol=rtol)
    r.set_initial_value(y0, tint[0])
    dt=(tint[1]-tint[0])/10000
    t1 = tint[1]
    tt=[]
    rr=[]
    while r.successful() and r.t < t1:
        tt.append(r.t+dt)
        rr.append(np.real(r.integrate(r.t+dt)))
    return np.array(tt),np.array(rr)

In [8]:
# Read in daily infection, mobility and mask use
# using a function at the end of this file.
# You could choose to copy and paste that function into a separate file
dailyinfection=read_covid_data(datapath+'daily_infections.csv',stateid);
mobility=read_covid_data(datapath+'mobility.csv',stateid);
mask=read_covid_data(datapath+'mask_use.csv',stateid);

# synchronize the timetables so they line up in time
data=dailyinfection.join(mobility,how='outer',lsuffix='_dailyinfection',rsuffix='').join(mask,how='outer',lsuffix='_mobility',rsuffix='_mask');

# select the time range for the analysis
# the datasets contain data after 2020-09-21 but those are projections at
# the time the paper was written.
TR = pd.date_range('2020-02-01','2020-09-21');
data=data.loc[TR];

# ---------get your ODE solution here

# -----start the fit from March 11, 2020
# beta estimated very early in the pandemic may be subject to 
# large uncertainties, as imported cases may be important compared to
# community spread
mask=data.mean_mask[39:];
mobility=data.mean_mobility[39:];


In [9]:
type(dailyinfection)

pandas.core.frame.DataFrame

In [22]:
x=dailyinfection.to_numpy()

In [23]:
type(x)

numpy.ndarray